In [1]:
# Set root package root directory.
import os
import sys
import pandas
import seaborn as sns

# Set for inline graph.
import matplotlib.pyplot as plt
%matplotlib inline

# Setup environment values.
os.environ['ML_FOR_SG_ROOT'] = 'Z:\Dev\Github\ml-for-sg'
sys.path.append('Z:\Dev\Github\ml-for-sg')

In [2]:
from lib.source_generator.shotgun.source_generator import ShotgunSourceGenerator
shotgun_source_generator = ShotgunSourceGenerator()
shotgun_source_generator.generate_source_data(project_id=70)

{'df_x_full':                 assets__acorn  assets__alice  assets__anders  \
 bunny_010_0010              0              0               0   
 bunny_010_0040              1              0               0   
 bunny_010_0050              0              0               1   
 bunny_010_0020              0              1               0   
 bunny_010_0030              0              0               0   
 
                 assets__backdrop  assets__banning  assets__billowy  \
 bunny_010_0010                 1                1                1   
 bunny_010_0040                 0                0                0   
 bunny_010_0050                 0                0                0   
 bunny_010_0020                 0                0                0   
 bunny_010_0030                 0                1                0   
 
                 assets__blue  assets__branch  assets__bunny  assets__bus  \
 bunny_010_0010             0               0              1            0   
 bunny_010_00

In [3]:
# Analyze feature based on source data.
from lib.resource.feature_analyzer import FeatureAnalyzer
feature_analyzer = FeatureAnalyzer()
feature_analyzer.analyze()

In [4]:
from lib.resource.resource_handler import ResourceHandler
resource_handler = ResourceHandler()
resource_handler.convert_source_to_resource()

Z:\Dev\Github\ml-for-sg\lib\resource\resource_handler.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_with_string_columns[new_column_name] = replaced_values


In [5]:
resource_handler = ResourceHandler()
data = resource_handler.load_resource_data()

In [6]:
df_x_full = data['df_x_full']
df_x_text_full = data['df_x_text_full']
df_y_full = data['df_y_full']

In [7]:
# Process
# 1. Grab all information from shotgun.
# 2. Generate dataframe of cost, df of features except text, df of text features.
# 3. Analyze #2 dataframe and generate recommand skip feature list.
# 4. Save skip_feature list based on #3 and analyzed data.
# 5. Select hot_keyword list based on #3 and analyzed data.
# 6. Generated resource data (merge text + configure) and convert to number and save.
# 7. Based on #6, trainer train several models.
# 8. Pick one model and generate report.
# 9. Save trained model.
# 10. make expectation based on model.

In [8]:
from lib.trainer.trainer import Trainer
trainer = Trainer()
merged_feature = trainer.merge_features(data)

C:\Users\KIHANG\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
from lib.model.linear_regression.lasso_model import LassoModel
lasso_model = LassoModel()
trained_lasso_model = trainer.train_model(lasso_model, merged_feature)
trainer.save_trained_model('lasso', trained_lasso_model)

In [10]:
trained_model = resource_handler.load_trained_model('lasso')

In [11]:
from collections import OrderedDict

new_data = OrderedDict([ 
    ('assets__backdrop', 1),
    ('assets__cliff', 0),
    ('sg_sequence__bunny_010', 1)]
)

c = ['assets__backdrop', 'assets__cliff', 'sg_sequence__bunny_010']
v = [1, 0, 1]

for column in merged_feature.columns:
    if column not in c:
        c.append(column)
        v.append(0)

# .values.reshape(1, -1) because it must be 2-dim, because we passed only one new observation
new_data = pandas.DataFrame([v], columns=c, index=['xxx']) 
# Use the model to make predictions
print new_data

     assets__backdrop  assets__cliff  sg_sequence__bunny_010  assets__acorn  \
xxx                 1              0                       1              0   

     assets__billowy  assets__blue  assets__bunny  assets__butterfly  \
xxx                0             0              0                  0   

     assets__clouds  assets__crow     ...      text__very  text__was  \
xxx               0             0     ...               0          0   

     text__way  text__we  text__what  text__wit  text__with  text__ye  \
xxx          0         0           0          0           0         0   

     text__you  text__your  
xxx          0           0  

[1 rows x 204 columns]


In [14]:
trained_model[0].predict(new_data)

array([ 12401.99447149])